**AIR QUALITY MONITORING**

**Background**

The objective of this project is to monitor and analyze the air quality trend in our environment specifically the Southeast Calgary area. Air quality readings were sourced from Calgary Region Airshed Zone (CRAZ) for the last 10 years (from 2014 - 2024) from the Calgary SE & Inglewood Craz's monitoring station.